In [ ]:
#general purpose packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import nltk
import re, string
!pip install emoji
import emoji
from collections import Counter


from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#transformers
!pip install transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras


#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

#set seed for reproducibility
seed=42

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.4 MB/s eta 0:00:00


Amazon Dataset

In [ ]:
Amazon = pd.read_parquet('/content/drive/Othercomputers/Laptop/2023 Spring 2/COMPSCI 760/amazon_electronics_other.parquet')
Amazon['reviewText'] = Amazon['reviewText'].astype(str)
Amazon['summary'] = Amazon['summary'].astype(str)
Amazon['reviews_text'] = Amazon['summary'] + ' ' + Amazon['reviewText']
Amazon.info()

In [ ]:
df, df_test = train_test_split(Amazon, test_size=0.9, random_state=63)
df.shape

(673959, 6)

In [ ]:
df_test.shape

(6065631, 6)

Kaggle Dataset

In [ ]:
Data = pd.read_csv('/content/drive/Othercomputers/Laptop/2023 Spring 2/COMPSCI 760/sample30.csv')

In [ ]:
Data['reviews_text'] = Data['reviews_text'].astype(str)
Data = Data[['reviews_text','user_sentiment','id']]
df, df_test = train_test_split(Data, test_size=0.1, random_state=63)
#df_test=Data
df.shape


(27000, 3)

In [ ]:
df_test.shape

(3000, 3)

In [ ]:
df.head()

,reviews_text,user_sentiment,id
29298,Smells extremely well paired with the shampoo....,Positive,AVpfW8y_LJeJML437ySW
25624,i was elated with this remake. The action scen...,Positive,AVpfPaoqLJeJML435Xk9
24718,I'm a big Godzilla Fan I've been watching then...,Positive,AVpfPaoqLJeJML435Xk9
8804,this product is so easy to use just open and u...,Positive,AVpf3VOfilAPnD_xjpun
14840,I have been very pleased with Clorox products ...,Positive,AVpf3VOfilAPnD_xjpun


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27000 entries, 29298 to 5164
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviews_text    27000 non-null  object
 1   user_sentiment  26999 non-null  object
 2   id              27000 non-null  object
dtypes: object(3)
memory usage: 843.8+ KB


In [ ]:
df.drop_duplicates(subset='reviews_text',inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24769 entries, 29298 to 5164
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviews_text    24769 non-null  object
 1   user_sentiment  24768 non-null  object
 2   id              24769 non-null  object
dtypes: object(3)
memory usage: 774.0+ KB


In [ ]:
##CUSTOM DEFINED FUNCTIONS TO CLEAN THE TWEETS


#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [ ]:
texts_new = []
for t in df.reviews_text:
    texts_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(t)))))

In [ ]:
texts_new_test = []
for t in df_test.reviews_text:
    texts_new_test.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(t)))))

In [ ]:
df['text_clean'] = texts_new
df_test['text_clean'] = texts_new_test

In [ ]:
df['text_clean'].head()

29298    smells extremely well paired with the shampoo ...
25624    i was elated with this remake the action scene...
24718    im a big godzilla fan ive been watching then s...
8804     this product is so easy to use just open and u...
14840    i have been very pleased with clorox products ...
Name: text_clean, dtype: object

In [ ]:
df_test['text_clean'].head()

15699    what a good idea order things that you need wh...
442      love it great hydrating skin feels awesome wou...
4464     i couldnt be more pleased with this vacuum cle...
19646    this product smells fabulous and is very effec...
27417    this is avery good movie for the kids but woul...
Name: text_clean, dtype: object

In [ ]:
df['text_clean'][1:8].values

array(['i was elated with this remake the action scenes were intense and the overall feel of the movie was the update i was looking for couple that with my favorite star godzilla in his fluffy state and well its worth owning i saw this in imax at the theatre and my family was not interested in going later watching the bluray they were like wow that was so much better than i thought it would be',
       'im a big godzilla fan ive been watching then since i was a child there have been many that i love and even moved me enough to cry and i would would put this right up there with some of my all time favorite i was not disappointed at the movie theater where i watched it twice and i have not been disappointed by my dvd purchase which i have also watched twice so far',
       'this product is so easy to use just open and use on all of your hard tables and chairs when ever you go out',
       'i have been very pleased with clorox products for over 10 years this review was collected as part o

In [ ]:
text_len = []
for text in df.text_clean:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

In [ ]:
df['text_len'] = text_len

In [ ]:
text_len_test = []
for text in df_test.text_clean:
    tweet_len = len(text.split())
    text_len_test.append(tweet_len)

In [ ]:
df_test['text_len'] = text_len_test

In [ ]:
df = df[df['text_len'] > 4]
df_test = df_test[df_test['text_len'] > 4]
print(f" DF SHAPE: {df.shape}")
print(f" DF TEST SHAPE: {df_test.shape}")

 DF SHAPE: (24285, 5)
 DF TEST SHAPE: (2934, 5)


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
token_lens = []

for txt in df['text_clean'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))

max_len=np.max(token_lens)

In [ ]:
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 512


In [ ]:
token_lens = []

for i,txt in enumerate(df['text_clean'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}")

INDEX: 1, TEXT: i was elated with this remake the action scenes were intense and the overall feel of the movie was the update i was looking for couple that with my favorite star godzilla in his fluffy state and well its worth owning i saw this in imax at the theatre and my family was not interested in going later watching the bluray they were like wow that was so much better than i thought it would be
INDEX: 9, TEXT: lots of redburgandy rice and black rice broth turns kind of purple after exposure but what a great flavor i mix it with wild blend also lundberg with a small amt of quinoa red or white near end of cook time a few herbs and a pc of carrot celery and onion and you have a meal we used this blend and added tacochili type seasonings and made tacos out of the rice as the meat there was no way to tell that no meat was in the product very good very meaty texture and really nice flavor with lots of chew a must try and a staple in my kitchen
INDEX: 12, TEXT: this conditioner actuall

In [ ]:
df['token_lens'] = token_lens

In [ ]:
df = df.sort_values(by='token_lens', ascending=False)
df.head(20)

In [ ]:
df = df.iloc[12:]
df.head()

,reviews_text,user_sentiment,id,text_clean,text_len,token_lens
4449,Hoover has cleaned our family's rugs for 3 gen...,Positive,AVpe9W4D1cnluZ0-avf0,hoover has cleaned our familys rugs for 3 gene...,390,444
17084,Brown rice (or hulled rice) is unmilled or par...,Positive,AVpf9pzn1cnluZ0-uNTM,brown rice or hulled rice is unmilled or partl...,359,421
26164,Been a huge Godzilla fan for as long as I can ...,Positive,AVpfPaoqLJeJML435Xk9,been a huge godzilla fan for as long as i can ...,380,421
20803,"To be honest, I grew up with an amazing old ca...",Positive,AVpfLnfPLJeJML434HpK,to be honest i grew up with an amazing old cas...,396,418
16932,Some of the best arborio I have used for risot...,Positive,AVpf9pzn1cnluZ0-uNTM,some of the best arborio i have used for risot...,353,417


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
token_lens_test = []

for txt in df_test['text_clean'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))

max_len=np.max(token_lens_test)

In [ ]:
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 512


In [ ]:
token_lens_test = []

for i,txt in enumerate(df_test['text_clean'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}")

INDEX: 2, TEXT: i couldnt be more pleased with this vacuum cleaner because it doesnt have all kinds of gadgets affixed to the arm it does not wear you out while using it because there is very little weight being transferred to the handle so the vacuum never gets to feeling heavy i have a back back and this makes a big difference it does a very fine job and is quiet the separate small canister vacuum has been easy to use and works well too
INDEX: 16, TEXT: this really is the only lip product i use i like the plum and neutral shades i dont need a mirror for the light shade and i need to always have something protecting my lips in colorado it never cakes or does anything uncomfortable so i dont need to fuss with it at all just keep reapplying it as it dries out in the sun we enjoy here this review was collected as part of a promotion
INDEX: 33, TEXT: have used this product for over 10 years even outside of the us when i lived in eu this new product is runny doesnt hold and smells terrible

In [ ]:
df_test['token_lens'] = token_lens_test
df_test = df_test.sort_values(by='token_lens', ascending=False)
df_test.head(10)
df_test = df_test.iloc[5:]
df_test.head(3)

,reviews_text,user_sentiment,id,text_clean,text_len,token_lens
21997,"This film in its acting, direction, photograph...",Positive,AVpfNfEMLJeJML434uf-,this film in its acting direction photography ...,418,469
15903,"I own a few shades of the Lip Shimmer: Fig, Rh...",Positive,AVpf63aJLJeJML43F__Q,i own a few shades of the lip shimmer fig rhub...,311,373
17150,To me this rice is so different than my regula...,Positive,AVpf9pzn1cnluZ0-uNTM,to me this rice is so different than my regula...,327,365


In [ ]:
df_test = df_test.sample(frac=1).reset_index(drop=True)

In [ ]:
df['user_sentiment'].unique()
df = df.dropna(subset=['user_sentiment']).copy()

In [ ]:
df_test['user_sentiment'].unique()
df_test = df_test.dropna(subset=['user_sentiment']).copy()

In [ ]:
df['user_sentiment'].value_counts()
df['user_sentiment'].unique()
df['user_sentiment'] = df['user_sentiment'].map({'Negative':0,'Positive':1})
df_test['user_sentiment'] = df_test['user_sentiment'].map({'Negative':0,'Positive':1})
df['user_sentiment'].value_counts()

1    21577
0     2695
Name: user_sentiment, dtype: int64

In [ ]:
df['user_sentiment'].unique()

array([1, 0])

In [ ]:
df_test['user_sentiment'].unique()

array([1, 0])

In [ ]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['text_clean']).reshape(-1, 1), np.array(df['user_sentiment']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text_clean', 'user_sentiment']);
train_os['user_sentiment'].value_counts()

1    21577
0    21577
Name: user_sentiment, dtype: int64

In [ ]:
X = train_os['text_clean'].values
y = train_os['user_sentiment'].values

In [ ]:
X_train, X_valid, y_train, y_valid= train_test_split(X, y ,test_size=0.5, stratify=y, random_state=seed)
ID_test = df_test['id'].values
X_test = df_test['text_clean'].values
y_test = df_test['user_sentiment'].values

In [ ]:
y_train_le = y_train.copy()
y_valid_le = y_valid.copy()
y_test_le = y_test.copy()
ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()
y_test = ohe.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()
print(f"TRAINING DATA: {X_train.shape[0]}\nVALIDATION DATA: {X_valid.shape[0]}\nTESTING DATA: {X_test.shape[0]}" )

TRAINING DATA: 21577
VALIDATION DATA: 21577
TESTING DATA: 2929


In [ ]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")
token_lens = []

for txt in X_train:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
for txt in X_test:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
max_length

492

In [ ]:
MAX_LEN=512

In [ ]:
def tokenize_roberta(data, max_len=MAX_LEN):
    input_ids = []
    attention_masks = []
    ids = []

    for i in range(len(data)):
        # 使用 tokenizer 对文本进行编码
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )

        # 截断或填充 input_ids 和 attention_masks 到相同长度
        input_id = encoded['input_ids'][:max_len]
        attention_mask = encoded['attention_mask'][:max_len]

        input_ids.append(input_id)
        attention_masks.append(attention_mask)

    # 将列表转换为NumPy数组，确保它们具有相同的长度
    input_ids_array = np.array(input_ids)
    attention_masks_array = np.array(attention_masks)

    return input_ids_array, attention_masks_array

In [ ]:
MAX_LEN

512

In [ ]:
train_input_ids, train_attention_masks = tokenize_roberta(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize_roberta(X_valid, MAX_LEN)
test_input_ids, test_attention_masks = tokenize_roberta(X_test, MAX_LEN)

In [ ]:
Counter(len(arr) for arr in train_input_ids)

Counter({512: 21577})

In [ ]:
Counter(len(arr) for arr in val_input_ids)


Counter({512: 21577})

In [ ]:
Counter(len(arr) for arr in test_input_ids)


Counter({512: 2929})

In [ ]:
Counter(len(arr) for arr in train_attention_masks)


Counter({512: 21577})

In [ ]:
Counter(len(arr) for arr in val_attention_masks)


Counter({512: 21577})

In [ ]:
Counter(len(arr) for arr in test_attention_masks)

Counter({512: 2929})

In [ ]:
len(ID_test)

2929

Check the test data index

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2929 entries, 0 to 2928
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviews_text    2929 non-null   object
 1   user_sentiment  2929 non-null   int64 
 2   id              2929 non-null   object
 3   text_clean      2929 non-null   object
 4   text_len        2929 non-null   int64 
 5   token_lens      2929 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 137.4+ KB


In [ ]:
acc=accuracy_score(X_test,df_test.loc[df_test['id'].isin(ID_test), 'text_clean'])
print(f'Accuracy:{acc}')

Accuracy:1.0


In [ ]:

test_input_idscp = test_input_ids.copy()
test_attention_maskscp = test_attention_masks.copy()
# 初始化一个空列表，用于存储展平后的数据
test_input_ids1 = np.array([np.array(item) for item in test_input_idscp])
test_attention_masks1 = np.array([np.array(item) for item in test_attention_maskscp])


In [ ]:
def CleantheTest(data_array):
    # 计算每个数组的长度并存储频数
    length_counts = Counter(len(arr) for arr in data_array)

    # 找到频数最多的长度
    most_common_length, most_common_count = length_counts.most_common(1)[0]

    # 保留具有频数最多长度的所有数组，并获取它们的原始数据中的索引
    filtered_data = [(arr, index) for index, arr in enumerate(data_array) if len(arr) == most_common_length]

    # 将 filtered_data 转换为 NumPy 数组
    filtered_data_array = np.array([arr for arr, _ in filtered_data])

    # 获取原始数据中的索引的数组
    original_indices = np.array([index for _, index in filtered_data])

    return original_indices,filtered_data_array

In [ ]:
#test_input_ids2=CleantheTest(test_input_ids1)
#test_attention_masks2=CleantheTest(test_attention_masks1)

In [ ]:
def create_model(bert_model, max_len=MAX_LEN):
    initial_learning_rate = 1e-5  # 初始学习率
    decay_steps = 1000  # 衰减步数
    decay_rate = 0.9  # 衰减率

    # 创建学习率衰减策略
    learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps, decay_rate)

    # 使用学习率策略创建优化器
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()

    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(opt, loss=loss, metrics=accuracy)
    return model
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [ ]:
model = create_model(roberta_model, MAX_LEN)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 tf_roberta_model (TFRobert  TFBaseModelOutputWithPooli   1246456   ['input_1[0][0]',             
 aModel)                     ngAndCrossAttentions(last_   32         'input_2[0][0]']             
                             hidden_state=(None, 512, 7                                           
                             68),                                                             

In [ ]:
history_2 = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=4, batch_size=32)

Epoch 1/4
237/675 [=========>....................] - ETA: 6:26:57 - loss: 0.4270 - categorical_accuracy: 0.7939

In [ ]:
result_roberta = model.predict([test_input_ids,test_attention_masks])
y_pred_roberta =  np.zeros_like(result_roberta)
y_pred_roberta[np.arange(len(y_pred_roberta)), result_roberta.argmax(1)] = 1

In [ ]:
print('\tClassification Report for RoBERTa:\n\n',classification_report(y_test,y_pred_roberta, target_names=['Negative', 'Positive']))

In [ ]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test,y_pred_roberta)
print(f'Accuracy:{acc}')

In [ ]:
Y_pred_roberta = np.argmax(y_pred_roberta, axis=1)
result_roberta = pd.DataFrame(columns=["id","Y_pred_roberta"])
result_roberta['id'] = ID_test
result_roberta['Y_pred_roberta'] = Y_pred_roberta
result_roberta.to_csv('/content/result91.csv',index=False)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix


# 计算混淆矩阵
confusion = confusion_matrix(np.argmax(y_test, axis=1),np.argmax(y_pred_roberta, axis=1))

# 计算精确度、召回率和 F1 分数
precision = precision_score(np.argmax(y_test, axis=1),np.argmax(y_pred_roberta, axis=1))
recall = recall_score(np.argmax(y_test, axis=1),np.argmax(y_pred_roberta, axis=1))
f1 = f1_score(np.argmax(y_test, axis=1),np.argmax(y_pred_roberta, axis=1))

print("Precision:", precision,"\nRecall:", recall,"F1 Score:", f1)